# CAPSTONE DATA COLLECTION PROCESS (MSI)
===================================

*Thank you for helping us out. We appreciate it!*

===================================

### INSTRUCTIONS:

#### STEP 1: Download the ipynb file from your inbox. Do not rename it. You will also find an excel file in the email with your assigned tasks (by way of serial numbers). Open this file in another tab (you will need it later).
#### STEP 2: Locate the ipynb file and place it on the desktop. 
#### STEP 3: Create a folder on the desktop. Name it "CAPSTONE MSI".
#### STEP 4: Save the ipynb file in the new folder and launch it on jupyter notebook.  


In [1]:
#getting current working directory - it should have "...desktop/CAPSTONE MSI" in it
#
#
#
import os
os.chdir('/Users/aanchalkhanna72/Desktop/')
os.getcwd()

'/Users/aanchalkhanna72/Desktop'

#### STEP 5: Run the 1st cell to  import the libraries. 

In [2]:
#Run this cell
#
#
#
import requests
import bs4
from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd
import concurrent.futures
import time
from timeit import default_timer as timer

#### STEP 6: Run the 2nd and 3rd cells. Input your serial number. Be very careful to not alter any code. Your output in cell 2 should correspond with  your first task in the file.

In [3]:
#Run this cell
#
#
#
data = pd.read_csv('https://raw.githubusercontent.com/Halaarav/CapStone/main/List.csv')
def serial(serial_number):
    temp = data.loc[data['Serial_Number'] == serial_number]
    year = int(temp.Year)
    month = int(temp.Month)
    start = int(temp.R1)
    end = int(temp.R2)
    return [year,month,start,end]

# THIS IS THE MOST IMPORTANT STEP.

In [4]:
#Input your serial number in the brackets with (****) and run this cell.
#
#
#
year,month,start,end = serial(176)

In [5]:
year,month,start,end 

(2008, 11, 39753, 39783)

#### STEP 7: Run the 4th cell.

In [6]:
#Run this cell 
#
#
#
#Given url for month, find links of all days published and store in the "day_links" variable, all articles in "articles_links" variable
#
#
#
#creating a loop for all articles to be scraped for each day in the month 
day_links=[]

for i in range(int(start),int(end)):

    #considering the url of month of 'august' that increments by 1, this loop will scrape articles for all days in the month of August
    day_link="https://economictimes.indiatimes.com/archivelist/year-"+str(year)+",month-"+ str(month)+",starttime-"+str(i)+".cms"
    day_links.append(day_link)    

def transform(day_link):
        
        #function to request browser to get information from given web page
        def get_url(url):
            return requests.get(url, {'headers':headers})
        
        #specifies the header/user agent,ie, any software that retrieves and presents Web content for end users or is implemented using Web technologies. User agents include Web browsers, media players, and plug-ins that help in retrieving, rendering and interacting with Web content.
        headers={
            'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 12_5_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6 Safari/605.1.15'
        }
        
        #creating empty dictionary where scraped data will be stored
        dict_day = {'date_published':[],'date_updated':[],'time':[],
                             'headline':[],'sector/category':[],'synopsis':[],'full_text':[]}
        
        #creates a list (not set) of all links scraped from archive of 1 day 
        article_links = []
        
        #creating 1st soup oject
        soup1=bs4.BeautifulSoup(get_url(day_link).text,'html.parser') #creates soup object for 1 day archive traversal
        #https://economictimes.indiatimes.com/archivelist/year-2008,month-1,starttime-39448.cms

        #date is scraped from the day page itself instead of going into individual news links
        try:
            date_pub=soup1.find_all('b')[1].text
        except:
            date_pub='NA'

        #collecting links of all news articles published on a day
        for ultag in soup1.find_all('ul', class_= 'content'): #looks for tag 'ul',class='content' under which links are present
            for litag in ultag.find_all('li'):
                for atag in litag.find_all('a'):
                    if(atag.get('href')!='#'):
                        link_reqd = "https://economictimes.indiatimes.com" + atag.get('href')
                        article_links.append(link_reqd)


        #Begins scraping content that needs to be written in the file

        #Scrapes article level data from links collected for the day stored in all_links
        for links in article_links: 

            #published date is appended as many times as there are number of article links for the day
            dict_day['date_published'].append(date_pub)

            #creating soup object of each link so that we can traverse through each news article
            soup=bs4.BeautifulSoup(get_url(links).text,features='html.parser')

            #scrapes headline, synopsis, date_updated, time, sector, full_text from respective HTML class 
            try:
                dict_day['headline'].append(soup.find('h1').text)
            except:
                dict_day['headline'].append('NA')

            #to prevent errors in non-existent updated dates/different synopsis class names
            try:
                dict_day['synopsis'].append(soup.find('h2',class_="summary").text)
            except:
                dict_day['synopsis'].append('NA')

            try:
                dict_day['date_updated'].append(" ".join([str(item) for item in soup.find('time',class_="jsdtTime").text.rsplit(' ')[2:5]])) #first splits elements, then merges them to give date for last updated 
            except:
                dict_day['date_updated'].append('NA')

            try:   
                dict_day['time'].append(" ".join([str(item) for item in soup.find('time',class_="jsdtTime").text.rsplit(' ')[5:8]])) #first splits elements, then merges them to give time for last updated
            except:
                dict_day['time'].append('NA')

            try:
                dict_day['sector/category'].append(links.rsplit('/')[4:6]) #gets sectors/category from the url itself
            except:
                dict_day['sector/category'].append('NA')
                
            try:
                try:
                    partial_text=soup.find('article',class_='artData clr').text
                    all_text=partial_text[:partial_text.find("Experience Your Economic Times")-21]
                    dict_day['full_text'].append(all_text)

                except:
                    some_text=soup.find('article',class_='artData clr paywall').text
                    final_text=some_text[:some_text.find("Experience Your Economic Times")-23]
                    dict_day['full_text'].append(final_text)
                    #partial_text_json=soup.find_all('script',type="application/ld+json")[1].get_text()
                    #all_text_json=partial_text_json[partial_text_json.find("articleBody")+14:partial_text_json.find("image")-15]
                    #dict_day['full_text'].append(all_text_json)
            
            except AttributeError:
                dict_day['full_text'].append('NA')
        
        #convert dictionary to a dataframe so as to save as csv
        df=pd.DataFrame(dict_day)
        #print(dict_day)

        #file name is unique as it's linked to unique article id visible in the url
        file_name=date_pub

        #Saving File as 'article id'.csv
        df.to_csv('./{}.csv'.format(file_name), sep=',')

        #empty dictionary and list
        dict_day.clear()
        article_links.clear()
        time.sleep(3)
        
        return ("Download done.")
 

In [7]:
day_links

['https://economictimes.indiatimes.com/archivelist/year-2008,month-11,starttime-39753.cms',
 'https://economictimes.indiatimes.com/archivelist/year-2008,month-11,starttime-39754.cms',
 'https://economictimes.indiatimes.com/archivelist/year-2008,month-11,starttime-39755.cms',
 'https://economictimes.indiatimes.com/archivelist/year-2008,month-11,starttime-39756.cms',
 'https://economictimes.indiatimes.com/archivelist/year-2008,month-11,starttime-39757.cms',
 'https://economictimes.indiatimes.com/archivelist/year-2008,month-11,starttime-39758.cms',
 'https://economictimes.indiatimes.com/archivelist/year-2008,month-11,starttime-39759.cms',
 'https://economictimes.indiatimes.com/archivelist/year-2008,month-11,starttime-39760.cms',
 'https://economictimes.indiatimes.com/archivelist/year-2008,month-11,starttime-39761.cms',
 'https://economictimes.indiatimes.com/archivelist/year-2008,month-11,starttime-39762.cms',
 'https://economictimes.indiatimes.com/archivelist/year-2008,month-11,starttime-

#### STEP 8: Run the 5th cell.

# DO NOT INTERRUPT WHILE IT IS RUNNING!!!!!

In [8]:
#Run this cell
#
#
#
#parallelizing operations
#from concurrent.futures import ThreadPoolExecutor
#start=timer()
#with ThreadPoolExecutor() as executor:
   # try:
#     for result in executor.map(transform,day_links):
 #           print(result)
  #  except:
   #     print("A task failed!")
#end=timer()
#print(((end-start)/60),"mins")

In [9]:
from joblib import Parallel, delayed

start=timer()
res = Parallel(
    n_jobs=-1
)(delayed(transform)(x) for x in day_links)
print(res)
end=timer()
print(((end-start)/60),"mins")
#alerts when code is done
#£duration = 3  # seconds
#freq = 440  # Hz
#os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))

['Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.', 'Download done.']
30.495880791216667 mins


#### STEP 9: Keep checking your desktop folder to see if the files (saved as "DATE-MONTH-YEAR.csv") are getting downloaded. Once the kernel is running, please ask one of us to take a look. DO NOT  INTERRUPT THE KERNEL.
#### STEP 10: Once the first task is COMPLETE AND YOU SEE A TIME STAMP in the last kernel, leave out the ipynb file and combine the remaining files in a new folder named "MONTH-YEAR". Zip this folder.
#### STEP 11: SUBMIT THIS ZIPPED FOLDER. 
#### STEP 12:  Repeat all steps from 5-11 in this notebook. Now, save the new folder named "(new) MONTH-YEAR". Remove ipynb file from folder. Zip the folder with combined months.
### SUBMIT!!!  YOU ARE DONE. THANK YOU!!
